In [5]:
from tqdm import tqdm
import json
import pandas as pd
from pathlib import Path
from typing import List, Dict, Any, Optional

class JsonToCsvProcessor:
    """
    一個用於處理 JSON 檔案並轉換為 CSV 的類別。

    該類別會遞迴遍歷來源目錄，**並自動過濾掉 macOS 的系統檔案**，
    讀取所有有效的 .json 檔案，提取特定鍵值，並將轉換後的路徑寫入 CSV。
    """
    def __init__(self, input_dir: str, output_csv: str, target_key: str = "台羅"):
        self.input_path = Path(input_dir)
        self.output_path = Path(output_csv)
        self.target_key = target_key
        self.processed_data: List[Dict[str, Any]] = []

        if not self.input_path.is_dir():
            raise FileNotFoundError(f"錯誤：來源目錄不存在 -> {self.input_path}")

    def _transform_path(self, json_path: Path) -> str:
        """
        根據指定規則，將 JSON 路徑轉換為目標 WAV 路徑 (此方法無需修改)。
        """
        path_str = str(json_path.resolve())
        transformed_path_str = path_str.replace("standard_json", "standard", 1)
        final_path = Path(transformed_path_str).with_suffix(".wav")
        return str(final_path)

    def _process_single_file(self, file_path: Path) -> Optional[Dict[str, Any]]:
        """
        處理單一 JSON 檔案 (此方法無需修改)。
        """
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                content = json.load(f)
            
            if self.target_key in content:
                target_path = self._transform_path(file_path)
                return {
                    self.target_key: content[self.target_key],
                    "pathfile": target_path
                }
            else:
                print(f"[警告] 檔案 {file_path.name} 中找不到鍵值 '{self.target_key}'，已略過。")
                return None
        except (json.JSONDecodeError, UnicodeDecodeError) as e: # 增加對 UnicodeDecodeError 的捕獲以提高穩健性
            print(f"[警告] 讀取檔案 {file_path.name} 時發生錯誤: {e}，已略過。")
            return None
        except Exception as e:
            print(f"[錯誤] 處理檔案 {file_path.name} 時發生未知錯誤: {e}")
            return None

    def run(self) -> None:
        """
        執行整個處理流程，增加對系統檔案的過濾。
        """
        print(f"[*] 開始遞迴掃描目錄: {self.input_path}")
        all_files = list(self.input_path.rglob('*.json'))
        
        if not all_files:
            print("[!] 目錄及其子目錄中未找到任何 .json 檔案。")
            return
            
        print(f"[*] 共找到 {len(all_files)} 個符合 .json 模式的檔案。")

        # --- 核心修改處：過濾掉以 '._' 開頭的 macOS 系統檔案 ---
        json_files = [f for f in all_files if not f.name.startswith('._')]
        
        print(f"[*] 過濾掉系統檔案後，將處理 {len(json_files)} 個有效 JSON 檔案...")

        for file_path in tqdm(json_files, desc="正在處理JSON檔案"):
            result = self._process_single_file(file_path)
            if result:
                self.processed_data.append(result)
        
        if self.processed_data:
            self._save_to_csv()
        else:
            print("[!] 沒有成功處理任何資料，不產生 CSV 檔案。")

    def _save_to_csv(self) -> None:
        """
        將處理完成的資料儲存為 CSV 檔案 (此方法無需修改)。
        """
        df = pd.DataFrame(self.processed_data)
        column_order = [self.target_key, "pathfile"]
        df = df[column_order]
        
        try:
            self.output_path.parent.mkdir(parents=True, exist_ok=True)
            df.to_csv(self.output_path, index=False, encoding='utf-8-sig')
            print(f"\n[+] 處理完成！資料已成功儲存至: {self.output_path}")
        except Exception as e:
            print(f"[錯誤] 儲存 CSV 檔案時發生錯誤: {e}")

# --- 主程式執行區塊 (無需修改) ---
if __name__ == "__main__":
    SOURCE_DIRECTORY = "./standard_json"
    OUTPUT_CSV_FILE = "./output/final_audio_paths.csv"
    
    try:
        processor = JsonToCsvProcessor(
            input_dir=SOURCE_DIRECTORY, 
            output_csv=OUTPUT_CSV_FILE
        )
        processor.run()
    except FileNotFoundError as e:
        print(e)
    except Exception as e:
        print(f"程式執行時發生未預期的錯誤: {e}")

[*] 開始遞迴掃描目錄: standard_json
[*] 共找到 163448 個符合 .json 模式的檔案。
[*] 過濾掉系統檔案後，將處理 81724 個有效 JSON 檔案...


正在處理JSON檔案: 100%|██████████| 81724/81724 [07:41<00:00, 177.25it/s] 



[+] 處理完成！資料已成功儲存至: output\final_audio_paths.csv


In [4]:
pip install tqdm

Note: you may need to restart the kernel to use updated packages.
